<h3>Given:</h3>
The data contains features extracted from the silhouette of
vehicles in different angles. Four "Corgie" model vehicles were
used for the experiment: a double decker bus, Cheverolet van,
Saab 9000 and an Opel Manta 400 cars. This particular
combination of vehicles was chosen with the expectation that
the bus, van and either one of the cars would be readily
distinguishable, but it would be more difficult to distinguish
between the cars.
<h3>Objective:</h3>
Apply the dimensionality reduction technique – PCA and train
a model using principal components instead of training the
model using just the raw data.

<h2>1. Reading the data</h2>

In [ ]:
# Importing the libraries
import pandas as pd        # for data manipulation
import seaborn as sns      # for statistical data visualisation
import numpy as np         # for linear algebra
import matplotlib.pyplot as plt      # for data visualization
from scipy import stats        # for calculating statistics

# Importing various machine learning algorithm from sklearn

from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score
from sklearn.metrics import mean_absolute_error,roc_curve,auc,accuracy_score
from scipy.stats import zscore
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D



In [ ]:
dataframe= pd.read_csv("vehicle.csv")  # Reading the data
dataframe.head()   # showing first 5 datas

In [ ]:
dataframe.shape

The data given has  19 columns and consist of 846 data. And all the data is read correctly.

In [ ]:
dataframe.info()

<h3>Obv</h3>
    The above information shows the following<br>
    a. The attributes are either int or float except class which is object <br>
    b. Further there maybe necessary to label encode class objects<br>
       <br><br>


In [ ]:
dataframe.isnull().sum()

There are many missing values. Should impute them.

In [ ]:
dataframe.apply(lambda x: len(x.unique()))

<h4>From  the data:</h4>

All variable have interval data except class attribute


In [ ]:
dataframe.describe()

<h3>We can infer that:</h3>
compactness is  normal distribution <br>
circularity is Slightly right skewed as mean and max have wide range gap<br>
distance_circularity as Left skewed<br>
skewness_about as right skewed<br>





In [ ]:
dataframe.fillna(dataframe.mean(),axis=0,inplace=True)

In [ ]:
dataframe.isnull().sum()

In [ ]:
dataframe.describe()

In [ ]:
dt=dataframe.iloc[:,0:18].copy()
dt

In [ ]:
sns.pairplot(dt)

Most of these columns share a strong correlation with each other. <br>
Both negative and positive correlation can be seen among the data<br>
We can also notice certain outliers in this data.

In [ ]:
dataframe['class'].value_counts()

<h3>Class will be our target column<h3>

In [ ]:
f = plt.subplots(1, figsize=(12,6))

colors = ["#FA5858", "#64FE2E","#ac3e9a"]
labels ="Car", "Bus","Van"

plt.suptitle('Information on Term Deposits', fontsize=20)

plt.pie(dataframe['class'].value_counts(),labels=labels,explode=(0,0.1,0.15),shadow=True,colors=colors, startangle=25,autopct='%1.1f%%')

plt.show()

There is a data imbalance as car has a significantly higher count as compared to bus and van.The count of car is higher as because there are 2 type of car present. And also we can see van has the lowest count.

In [ ]:
dataframe.skew()

In [ ]:
plt.figure(figsize=(10,10))
plt.subplot(6,1,1)
sns.boxplot(dataframe.compactness)
plt.subplot(6,1,2)
sns.boxplot(dataframe.circularity)
plt.subplot(6,1,3)
sns.boxplot(dataframe.distance_circularity)
plt.subplot(6,1,4)
sns.boxplot(dataframe.radius_ratio)
plt.subplot(6,1,5)
sns.boxplot(dataframe["pr.axis_aspect_ratio"])
plt.subplot(6,1,6)
sns.boxplot(dataframe["scaled_radius_of_gyration.1"])

In [ ]:
plt.figure(figsize=(10,10))
plt.subplot(6,1,1)
sns.distplot(dataframe.compactness)
plt.subplot(6,1,2)
sns.distplot(dataframe.circularity)
plt.subplot(6,1,3)
sns.distplot(dataframe.distance_circularity)
plt.subplot(6,1,4)
sns.distplot(dataframe.radius_ratio)
plt.subplot(6,1,5)
sns.distplot(dataframe["pr.axis_aspect_ratio"])
plt.subplot(6,1,6)
sns.distplot(dataframe["scaled_radius_of_gyration.1"])


<h3>Obv</h3>
<li>Compactness column shows an approximately normal distribution curve while having no outliers.</li>

<li>The distribution plot of the circularity column has 3 peaks and is right skewed.</li><li>Distance Circularity is left skewed and has 2 peaks in the plot.
</li>
<li>Radius ratio has outliers and is right screwed a bit</li>
<li>Pr.Axis aspect ratio is approx normally distributed as well as right skewed and there is presence of outliers.</li>
<li> Scaled_radius_of_gyration.1 approximatly normal distribution with no outliers.</li>


In [ ]:
fig,(a1,a2)=plt.subplots(nrows = 1, ncols = 2, figsize = (13, 5))
sns.boxplot(x = 'hollows_ratio', data = dataframe, orient = 'v', ax = a1)
sns.distplot(dataframe.hollows_ratio, ax = a2)

Hollow_ratio has 2 peaks and no outliers in the data.

In [ ]:
dataframe["class"].hist(bins=3)

As in the data, the count of car is more compared to bus and van. Due to which there maybe chances that the model perdiction will be effected due to this.<br>
<h2>Checking the influence of various attributes on customer subscribing term deposit </h2>

Here we can see some attributes are positively correlated (ex. circularity and max.length_rectangularity)<br>
negatively correlated (elongatedness and scatter_ratio<br>
and no correlation at all (compactness and skewness_about.1)
<br>Checking the effect of these variables om target i.e class attribute
<h3>Influence of Elongatedness & scatter_ratio on Target</h3>



In [ ]:
plt.figure(figsize=(8,8))
sns.scatterplot(x='elongatedness',y='scatter_ratio' ,data=dataframe,hue='class')


<h4>Obv:</h4>  Looks like bus has lesser elongatedness  but scatter ratio is highest<br>
van is quite opposite to bus where elongatedness is higher and scatter ratio  is in lesser range<br>
<h3>Influence of circularity & max.length_rectangularity on target
</h3>


In [ ]:
fig,(a1,a2)=plt.subplots(nrows = 2, ncols = 1, figsize = (13, 15))
sns.boxplot(x='class',y='max.length_rectangularity',data=dataframe,ax=a1)
sns.scatterplot(x='circularity',y='max.length_rectangularity',hue='class',data=dataframe,ax=a2)


<h4>Obv</h4>a. car has a wide range of cicularity and also max.length_rectangularity.<br>
b. Whereas the bus and van do not.<br>
c. So cicularity and max.lenght_rectangularity can be used to classify mainly bus and van but not cars


<h3>Influence of scaled_radius_of_gyration.1 on class
</h3>

In [ ]:
plt.figure(figsize=(10,10))
sns.catplot(x="class",y='scaled_radius_of_gyration.1', data=dataframe)

<h3>Obv</h3>
We can see that scaled_radius_of_gyration.1 cant only use alone to classify the vehicles.

<h3>Influence of circularity and distance_circularity on class
</h3>

In [ ]:
sns.lineplot(x='circularity',y='distance_circularity',data=dataframe,hue='class')


<h4>Obv</h4>
The distance_circularity increases as circularity increases for all the class of vehicles.
<br>
also we can infer that the van has lesser cirularity and distance_circularity than others



<h2>Corelation of Attributes</h2>

In [ ]:
corelation=dt.corr()
corelation

In [ ]:
plt.figure(figsize=(20,20))
a=sns.heatmap(corelation,annot=True)

<h3>Obv</h3>
<li>Here most of the column share high collinearity both negatively and positively. </li>
<li>And also we can see that elongatedness is most negatively corelated with almost all the data.</li>
<li>This should not be the case in the model.</li>
<li>As data are correlated, we can drop some of the columns which are corelated.</li>

<h2>Preparing the data for modeling</h2>



<h2>Classification Models</h2>


In [ ]:
dataframe.columns

In [ ]:
features=['compactness', 'circularity', 'distance_circularity', 'radius_ratio',
       'pr.axis_aspect_ratio', 'max.length_aspect_ratio', 'scatter_ratio',
       'elongatedness', 'pr.axis_rectangularity', 'max.length_rectangularity',
       'scaled_variance', 'scaled_variance.1', 'scaled_radius_of_gyration',
       'scaled_radius_of_gyration.1', 'skewness_about', 'skewness_about.1',
       'skewness_about.2', 'hollows_ratio']
X=dataframe[features]
Y=dataframe['class']         

Normalising the data

In [ ]:
X=X.apply(zscore)
X

Label Encoding the target "class" column where<br>
van is label encoded as 1 <br>
car is label encoded as 2 <br>
bus is label encoded as 3<br>

In [ ]:
Y.replace(['van','car','bus'],[1,2,3],inplace=True)
Y

<h3> Splitting the Data</h3>
Splitting the model in 7:3 ratio

In [ ]:
train_X,test_X,train_y,test_y=train_test_split(X,Y,test_size=0.3,random_state=1)
train_X.count() 

In [ ]:
train_X.head()

In [ ]:
test_X.count()

In [ ]:
test_X.head()

<h2>Using SVC for prediction</h2><br>
Trianing the model

In [ ]:
parameters={
    'C':[0.01,0.25,0.5,1],
    'kernel':['rbf','linear']
}
model=SVC()
best_SVC=GridSearchCV(model,param_grid=parameters,scoring='accuracy',cv=10)
best_SVC

Predicting from the trained model and showing the confusion matrix

In [ ]:
best_SVC.fit(train_X,train_y)

In [ ]:
best_SVC.best_params_

In [ ]:
svm_model=SVC(C=1,kernel='rbf',random_state=1)
svm_model

In [ ]:
svm_model=svm_model.fit(train_X,train_y)

In [ ]:
predict=svm_model.predict(test_X)
print(predict[0:1000])
metrics=confusion_matrix(test_y,predict)
metrics

In [ ]:
sns.heatmap(metrics,annot=True,fmt='g',cmap='Blues')

In [ ]:
print(classification_report(test_y,predict))

In [ ]:
svm_accuracy=accuracy_score(test_y,predict)
svm_accuracy

In [ ]:

svm_eval = cross_val_score(estimator = svm_model, X = train_X, y = train_y, cv = 10)
svm_eval.mean()

<h3>Obv:</h3>
The heat map shows that the model perdicts the outcome pretty good as the data are pretty much correlated.<br>
The f1 score is also looks fine (91%)<br>
As we can see the accuracy is 95.27% which indicates the model predicts well enough.<br>
The mean after the cross validation is also showing the better result 95.94%<br>
<br>
<h2> Using PCA with SVM model</h2>
<br>



Giving n_components as 18 i.e the number of columns (Except class)

In [ ]:
pca=PCA(n_components=18)
pca

Fitting the mondel the pca and displaying Eigen values

In [ ]:
pca.fit(X)
pca.explained_variance_

 Variance ratio will  be

In [ ]:
pca.explained_variance_ratio_

And the eigen vectors will be

In [ ]:
pca.components_

Bar plotting the eigen values

In [ ]:
sns.barplot(x=list(range(1,19)),y=pca.explained_variance_)

Elbow plotting the eigen values

In [ ]:
plt.plot(list(range(1,19)),pca.explained_variance_,'ro-', linewidth=2)
plt.title('Elbow Plot')
plt.xlabel('Principal Component')
plt.ylabel('Eigenvalue')

In [ ]:
plt.step(list(range(1,19)), np.cumsum(pca.explained_variance_ratio_), where = 'mid')
plt.ylabel('Cum of variation explained')
plt.xlabel('Eigen value')


We have 8 components in this model that explain 95% variation in this data.
<br>
So giving n_components as 8 and performing PCA

In [ ]:
pca=PCA(n_components=8)
pca.fit(X)

The eigen value, variance ratio will be

In [ ]:
pca.explained_variance_

In [ ]:
pca.explained_variance_ratio_

In [ ]:
pca_X = pca.transform(X)

In [ ]:
sns.pairplot(pd.DataFrame(pca_X))

So we can see that the correlation is removed among the data and is ready to be get trained on. <br>The data is reduced to 8 columns

Again splitting and training the model

In [ ]:
train_X,test_X,train_y,test_y=train_test_split(pca_X,Y,test_size=0.3,random_state=1)
pd.DataFrame(train_X).count()

<h3>Preparing SVC model with GridSearch for getting the best model</h3>

In [ ]:
parameters={
    'C':[0.01,0.25,0.5,1],
    'kernel':['rbf','linear']
}
model=SVC()
best_PCA_SVC_grid=GridSearchCV(model,param_grid=parameters,scoring='accuracy',cv=10)
best_PCA_SVC_grid

<h3>Fitting the model with gridsearch</h3>

In [ ]:
best_PCA_SVC_grid.fit(train_X,train_y)

Getting the best parameters for the model

In [ ]:
best_PCA_SVC_grid.best_params_

<h3> Trainig the model on the above parameters<h3>

In [ ]:
best_PCA_SVC=SVC(C=1,kernel='rbf',random_state=1)
best_PCA_SVC

Fitting the data in the model

In [ ]:
best_PCA_SVC=best_PCA_SVC.fit(train_X,train_y)

Predicting from the model

In [ ]:
predict=best_PCA_SVC.predict(test_X)
print(predict[0:1000])
metrics_pca=confusion_matrix(test_y,predict)
metrics_pca

In [ ]:
sns.heatmap(metrics_pca,annot=True,fmt='g',cmap='Blues')

In [ ]:
print(classification_report(test_y,predict))

In [ ]:
pca_svm_accuracy=accuracy_score(test_y,predict)
pca_svm_accuracy

In [ ]:

pca_svm_eval = cross_val_score(estimator = best_PCA_SVC, X = train_X, y = train_y, cv = 10)
pca_svm_eval.mean()

<h3>Obv:</h3>
The heat map shows that the model perdicts the outcome pretty good as the data are pretty much correlated.<br>
The f1 score is also looks fine (90%)<br>
As we can see the accuracy is 94.09% which indicates the model predicts well enough.<br>
The mean after the cross validation is also showing the better result 94.4%<br>

<h2>Comparig the model </h2><h4>with and without PCA</h4>

In [ ]:
data=[[svm_accuracy,svm_eval.mean()],[pca_svm_accuracy,pca_svm_eval.mean()]]

In [ ]:
compare=pd.DataFrame(data,columns=["Accuracy","Cross validation Mean"],index=["SVC","SVC with PCA"])
compare

In [ ]:
plt.subplot(2,2,1)
sns.heatmap(metrics,annot=True,fmt='g',cmap='Blues')
plt.subplot(2,2,2)
sns.heatmap(metrics_pca,annot=True,fmt='g',cmap='Blues')

<h3>We can see that SVM is good model for classification as it gives good accuracy score<br>
    And also Principal Companent Analysis is good for reducing the dimension of the dataset ( here from 18 to 8 ) with neglegible accuracy and cross validation score (nearly 1 % of decrease) which is good.